Run cells bellow to download and preprocess the **Steam-200K** dataset.

Estimated time is around 45 minutes.

In [1]:
#!unzip /home/ubuntu/vasp/Datasets/steam-200k.zip

Archive:  /home/ubuntu/vasp/Datasets/steam-200k.zip
   creating: preprocessed_100/
  inflating: preprocessed_100/iId2itemName.json  
  inflating: preprocessed_100/validation_te.csv  
  inflating: preprocessed_100/validation_tr.csv  
  inflating: preprocessed_100/unique_iId.txt  
  inflating: preprocessed_100/train_te.csv  
  inflating: preprocessed_100/train_tr.csv  
  inflating: preprocessed_100/test_tr.csv  
  inflating: preprocessed_100/test_te.csv  


In [3]:
import pandas as pd
import os

All Items

In [26]:
ds_path = "/home/ubuntu/vasp/Datasets/steam-200k/preprocessed_100"
ds_path_out = "/home/ubuntu/vasp/Datasets/steam-200k/preprocessed_vasp"

In [8]:
temp_tr = pd.read_csv(os.path.join(ds_path, 'train_tr.csv'))
temp_te = pd.read_csv(os.path.join(ds_path, 'train_te.csv'))
train = pd.concat([temp_tr, temp_te])
print(train.head())
print(len(train))

temp_tr = pd.read_csv(os.path.join(ds_path, 'validation_tr.csv'))
temp_te = pd.read_csv(os.path.join(ds_path, 'validation_te.csv'))
val = pd.concat([temp_tr, temp_te])
print(val.head())
print(len(val))

temp_tr = pd.read_csv(os.path.join(ds_path, 'test_tr.csv'))
temp_te = pd.read_csv(os.path.join(ds_path, 'test_te.csv'))
test = pd.concat([temp_tr, temp_te])
print(test.head())
print(len(test))

full = pd.concat([train, val, test])
print(full.head())
print(len(full))

   uId  iId                                        title
0  543  110                               Counter-Strike
1  543  532                    Call of Duty World at War
2  543  325                             Total War ATTILA
3  543  183  Call of Duty Modern Warfare 2 - Multiplayer
4  543  167                Call of Duty Modern Warfare 2
23203
    uId  iId                       title
0  1420   91             Team Fortress 2
1  1420   10  Sid Meier's Civilization V
2  1420    0  The Elder Scrolls V Skyrim
3  1420   23                   Far Cry 3
4  1420    3           Fallout New Vegas
2999
    uId  iId                                        title
0  1622  112                Counter-Strike Condition Zero
1  1622  183  Call of Duty Modern Warfare 2 - Multiplayer
2  1622  110                               Counter-Strike
3  1622  114                       Call of Duty Black Ops
4  1622   77                      Half-Life 2 Episode Two
2718
   uId  iId                                     

In [27]:
items = pd.read_csv(os.path.join(ds_path, 'unique_iId.txt'))
items.to_json(os.path.join(ds_path_out, 'items.json'))
items

,iId
0,0
1,1
2,2
3,3
4,4
...,...
2114,5131
2115,2971
2116,5152
2117,5065


In [23]:
result = train['iId'].isin(test['iId'])

result.unique()

array([ True, False])

Purchases and grouped purchases.

In [40]:
interactions = full
interactions['iId'] = interactions['iId'].apply(str)
interactions['userId'] = interactions['uId'].apply(str)
#interactions.to_json(os.path.join(ds_path_out, 'purchases.json'))
interactions['itemids'] = interactions[['userId','iId']].groupby(['userId'])['iId'].transform(lambda x: ','.join(x))
iii = interactions[['userId','itemids']].drop_duplicates()
print(iii.head())
iii.reset_index(drop=True, inplace=True)
iii.to_json(os.path.join(ds_path_out, 'purchases_txt.json'))
print(iii.head())

   userId                                            itemids
0     543  110,532,325,183,167,89,186,114,272,34,56,115,2...
12    797                            0,119,23,4,2086,218,276
18    591                             89,88,243,167,4,49,543
24    105  270,115,505,408,183,888,898,34,548,1931,4,232,...
60    953                        88,10,48,110,81,57,1597,246
  userId                                            itemids
0    543  110,532,325,183,167,89,186,114,272,34,56,115,2...
1    797                            0,119,23,4,2086,218,276
2    591                             89,88,243,167,4,49,543
3    105  270,115,505,408,183,888,898,34,548,1931,4,232,...
4    953                        88,10,48,110,81,57,1597,246


Users.

In [42]:
iii['userId'].to_frame().to_json(os.path.join(ds_path_out, 'users.json'))
print(iii['userId'].to_frame().head())

  userId
0    543
1    797
2    591
3    105
4    953


Items

In [16]:
#items[items.itemid.isin(purchases_item_count_pu5.itemid)].to_json("items_pu5.json")
items[items.itemid.isin(purchases_item_count_pu5.itemid)]

,itemid,product_name,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
27271,131250,No More School (2000),Comedy
27272,131252,Forklift Driver Klaus: The First Day on the Jo...,Comedy|Horror
27273,131254,Kein Bund für's Leben (2007),Comedy
27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy


Items sorted by number of interactions

In [17]:
#purchases_item_count_pu5.to_json("items_sorted_pu5.json")
purchases_item_count_pu5

,itemid,nr_of_purchases
6620,318,55705
6186,296,52265
12160,593,50061
7545,356,47302
4894,260,42578
...,...,...
3318,131050,1
3317,131027,1
3316,131023,1
3313,130996,1


Users sorted by number of interactions

In [18]:
#pu5.to_json("users_sorted_pu5.json")
pu5

,userid,nr_of_purchases
28626,125794,3177
118782,82418,2655
120595,8405,2503
20206,118205,2377
35395,131894,2349
...,...,...
4572,104116,5
28224,125431,5
11086,109989,5
67106,3584,5


Create train, val and test split.

In [19]:
users = pd.read_json('users_pu5.json')
shuffled_users = users.sample(frac=1., random_state=42)
test_users = shuffled_users.iloc[:10000]
val_users = shuffled_users.iloc[10000:20000]
train_users = shuffled_users.iloc[20000:]

#test_users.to_json("test_users.json")
print(test_users.head())
#val_users.to_json("val_users.json")
print(val_users.head())
#train_users.to_json("train_users.json") 
print(train_users.head())

len(train_users),len(val_users),len(test_users)

          userid
9483226    65567
18812883  130320
1125581     7677
5262991    36081
17249182  119275
          userid
15163263  104804
13123091   90664
11207540   77425
9316001    64428
13769119   95146
          userid
9141029    63177
17703308  122423
5668357    39001
1562218    10557
2448694    16563


(116677, 10000, 10000)

In [24]:
test_users.sort_index()

,userid
1192,11
1511,13
1770,14
5057,50
8149,73
...,...
19989248,138410
19991285,138422
19991592,138426
19994302,138449


List of generated json files with preprocessed data.

In [20]:
!ls *.json

VASP_ML20_1_loss.json	  purchases.json	  train_users.json
VASP_ML20_1_metrics.json  purchases_pu5.json	  users_pu5.json
items.json		  purchases_txt.json	  users_sorted_pu5.json
items_pu5.json		  purchases_txt_pu5.json  val_users.json
items_sorted_pu5.json	  test_users.json


In [21]:
tu=pd.read_json('train_users.json')


In [22]:
tu

,userid
10000069,69140
10000124,69141
10000172,69142
10000269,69143
10000322,69144
...,...
9999490,69133
9999604,69134
9999651,69135
9999705,69136
